### Import Dependencies

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
import torch
from torchvision import datasets, transforms, models  # datsets  , transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.nn.functional as F
from datetime import datetime

### Import Dataset

dataset = datasets.ImageFolder<b> Dataset Link (Plant Vliiage Dataset ):</b><br> <a href='https://data.mendeley.com/datasets/tywbtsjrjv/1'> https://data.mendeley.com/datasets/tywbtsjrjv/1 </a> 

In [5]:
transform2 = transforms.Compose([
    transforms.Resize((299, 299)),  
    transforms.ToTensor(),          
    transforms.Normalize(
        mean=[0.5, 0.5, 0.5],      
        std=[0.5, 0.5, 0.5]         
    )
])

In [6]:
transform3 = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [7]:
transform4 = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),          
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],  
        std=[0.229, 0.224, 0.225]    
    )
])

In [8]:
dataset = datasets.ImageFolder("/kaggle/input/mepco-tropic-leaf/MepcoTropicLeaf-V1/Database", transform=transform4)

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/kaggle/input/mepco-tropic-leaf/MepcoTropicLeaf-V1/Database'

In [7]:
dataset

Dataset ImageFolder
    Number of datapoints: 3777
    Root location: /kaggle/input/mepco-tropic-leaf/MepcoTropicLeaf-V1/Database
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [8]:
indices = list(range(len(dataset)))

In [9]:
split = int(np.floor(0.70 * len(dataset)))  # train_size

In [10]:
validation = int(np.floor(0.60 * split))  # validation

In [11]:
print(0, validation, split, len(dataset))

0 1585 2643 3777


In [12]:
print(f"length of train size :{validation}")
print(f"length of validation size :{split - validation}")
print(f"length of test size :{len(dataset)-validation}")

length of train size :1585
length of validation size :1058
length of test size :2192


In [13]:
np.random.shuffle(indices)

### Split into Train and Test

In [14]:
train_indices, validation_indices, test_indices = (
    indices[:validation],
    indices[validation:split],
    indices[split:],
)

In [15]:
train_sampler = SubsetRandomSampler(train_indices)
validation_sampler = SubsetRandomSampler(validation_indices)
test_sampler = SubsetRandomSampler(test_indices)

In [16]:
train_sampler

In [17]:
targets_size = len(dataset.class_to_idx)
print(targets_size)

50


In [18]:
num_classes_list = list(dataset.class_to_idx.values())
print(num_classes_list)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]


### Model

<b>Convolution Aithmetic Equation : </b>(W - F + 2P) / S + 1 <br>
W = Input Size<br>
F = Filter Size<br>
P = Padding Size<br>
S = Stride <br>

### Transfer Learning

In [19]:
model = models.vgg16(pretrained=True)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

In [20]:
model2=models.mobilenet_v2(pretrained=True)

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


  0%|          | 0.00/13.6M [00:00<?, ?B/s]

In [21]:
model3=models.resnet18(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [22]:
for params in model3.parameters():
   params.requires_grad = True

In [23]:
model3

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [24]:
n_features = model3.classifier[0].in_features
n_features

AttributeError: 'ResNet' object has no attribute 'classifier'

In [ ]:
n_features = model2.classifier[1].in_features #mobilenet case
n_features

In [ ]:
n_features = model3.conv1.in_channels #resnet18
n_features

In [ ]:
model3.classifier = nn.Sequential(
     nn.Linear(n_features, 1024),
     nn.ReLU(),
     nn.Dropout(0.4),
     nn.Linear(1024, targets_size),
 )

### Original Modeling

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
model3.to(device)

In [ ]:
pip install torchsummary

In [ ]:
from torchsummary import summary
summary(model3, (3, 299, 299))

In [ ]:
criterion = nn.CrossEntropyLoss()  # this include softmax + cross entropy loss
optimizer = torch.optim.Adam(model3.parameters())

### Batch Gradient Descent

In [1]:
def batch_gd(model, criterion, train_loader, test_laoder, epochs):
    train_losses = np.zeros(epochs)
    test_losses = np.zeros(epochs)

    for e in range(epochs):

        t0 = datetime.now()
        train_loss = []
        for inputs, targets in train_loader:
           
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            output = model(inputs)
            loss = criterion(output, targets)
            train_loss.append(loss.item())  # torch to numpy world
            loss.backward()
            optimizer.step()

        train_loss = np.mean(train_loss)

        validation_loss = []

        for inputs, targets in validation_loader:
            
            inputs, targets = inputs.to(device), targets.to(device)

            output = model(inputs)

            loss = criterion(output, targets)

            validation_loss.append(loss.item())  # torch to numpy world

        validation_loss = np.mean(validation_loss)

        train_losses[e] = train_loss

        dt = datetime.now() - t0

        print(f"Epoch : {e+1}/{epochs} Train_loss:{train_loss:.3f} Duration:{dt}")

    return train_losses

In [ ]:
batch_size = 64
train_loader = torch.utils.data.DataLoader(
    dataset, batch_size=batch_size, sampler=train_sampler
)
test_loader = torch.utils.data.DataLoader(
    dataset, batch_size=batch_size, sampler=test_sampler
)
validation_loader = torch.utils.data.DataLoader(
    dataset, batch_size=batch_size, sampler=validation_sampler
)

In [ ]:
data_transforms = {
    'train_loader': transforms.Compose([
        transforms.RandomAffine(degrees=10, translate=(0.05, 0.05), shear=5),
        transforms.ColorJitter(hue=0.05, saturation=0.05),
        transforms.RandomHorizontalFlip(),
        transforms.Grayscale(num_output_channels=1),
         transforms.RandomApply([transforms.GaussianBlur(kernel_size=7)], p=0.2),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,)),
    ]),
    'validation_loader': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,)),
    ])
}

In [2]:
train_losses = batch_gd(model3, criterion, train_loader, validation_loader, 10)

NameError: name 'model3' is not defined

In [ ]:
import torch
from torchvision import transforms
import matplotlib.pyplot as plt
data_iterator = iter(train_loader)
images, labels = next(data_iterator)

inverse_transform = transforms.Compose([
    transforms.Normalize(mean=[0, 0, 0], std=[1/0.5, 1/0.5, 1/0.5]),
    transforms.ToPILImage()  

image_to_show = inverse_transform(images[0]).convert("RGB")

plt.imshow(image_to_show)
plt.title(f"Label: {labels[4]}")  
plt.axis("off")  
plt.show()

### Save the Model

### Load Model

In [ ]:
# %matplotlib notebook

### Plot the loss

In [ ]:
plt.plot(train_losses , label = 'train_loss')
plt.xlabel('No of Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

### Accuracy

In [ ]:
def accuracy(loader):
    n_correct = 0
    n_total = 0
    model3.cuda()
    for inputs, targets in loader:
        inputs, targets = inputs.cuda(), targets.cuda()

        outputs = model3(inputs)
        
        #print(outputs)

        _, predictions = torch.max(outputs, 1)

        n_correct += (predictions == targets).sum().item()
        n_total += targets.shape[0]

    acc = n_correct / n_total
    return acc

In [ ]:
train_acc = accuracy(train_loader)
test_acc = accuracy(test_loader)
validation_acc = accuracy(validation_loader)

In [ ]:
print(
    f"Train Accuracy : {train_acc}\nTest Accuracy : {test_acc}\nValidation Accuracy : {validation_acc}"
)

In [ ]:
torch.save(model2, "PlantMed2.pt")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loaded_model = torch.load("PlantMed2.pt", map_location=device)
loaded_model.eval()

In [ ]:
from PIL import Image

In [ ]:
transforms = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
image_path = "/kaggle/input/leafwithlessrizz/SexyPlantLessDef.jpg"  
image = Image.open(image_path)

In [ ]:
plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.title("Original Image")
plt.imshow(image)
plt.axis('off')

In [ ]:
input_data = transforms(image).unsqueeze(0).to(device)

In [ ]:
with torch.no_grad():
    output = loaded_model(input_data)

In [ ]:
_, predicted_class = output.max(1)

In [ ]:
predicted_class

In [ ]:
class_to_idx = dataset.class_to_idx 
idx_to_class = {idx: class_name for class_name, idx in class_to_idx.items()}
index = 22
class_name = idx_to_class.get(index, "Unknown")  

print(f"Class name for index {index}: {class_name}")